# Aggressive Optimization for Small Configurations

Focus optimization effort on smaller configurations (n=1-50) since they contribute more to the score.

In [ ]:
import math
import random
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import time

getcontext().prec = 25
scale_factor = Decimal('1e15')
random.seed(42)
np.random.seed(42)

print("Libraries loaded")

In [ ]:
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

print("ChristmasTree defined")

In [ ]:
def get_side_length(trees):
    if not trees:
        return Decimal('0')
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor
    return max(maxx - minx, maxy - miny)

def calculate_score(side_lengths):
    return sum(float(s)**2 / n for n, s in side_lengths.items())

def has_any_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def has_collision(tree, other_trees):
    if not other_trees:
        return False
    polygons = [t.polygon for t in other_trees]
    tree_index = STRtree(polygons)
    indices = tree_index.query(tree.polygon)
    for idx in indices:
        if tree.polygon.intersects(polygons[idx]) and not tree.polygon.touches(polygons[idx]):
            return True
    return False

print("Helper functions defined")

In [ ]:
def calculate_bbox_side_at_angle(angle_deg, points):
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    if not trees:
        return Decimal('0'), 0.0
    
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points) / float(scale_factor)
    hull_points = points_np[ConvexHull(points_np).vertices]
    initial_side = calculate_bbox_side_at_angle(0, hull_points)

    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.001, 89.999), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun

    if initial_side - found_side > 1e-8:
        return Decimal(str(found_side)), found_angle_deg
    else:
        return Decimal(str(initial_side)), 0.0

def apply_rotation(trees, angle_deg):
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]

    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])

    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])

    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center / float(scale_factor)
    rotated = shifted.dot(rot_matrix.T) + rotation_center / float(scale_factor)

    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            str(rotated[i, 0]), 
            str(rotated[i, 1]),
            str(float(trees[i].angle) + angle_deg)
        )
        rotated_trees.append(new_tree)
    return rotated_trees

print("Rotation functions defined")

In [ ]:
def generate_weighted_angle():
    while True:
        angle = random.uniform(0, 2 * math.pi)
        if random.uniform(0, 1) < abs(math.sin(2 * angle)):
            return angle

def build_config_from_scratch(n, num_attempts=50):
    """Build a configuration from scratch with many attempts."""
    if n == 0:
        return [], Decimal('0')
    
    best_trees = None
    best_side = Decimal('Infinity')
    
    for attempt in range(num_attempts):
        placed_trees = []
        
        # First tree at origin with random angle
        first_angle = random.uniform(0, 360)
        placed_trees.append(ChristmasTree(angle=str(first_angle)))
        
        # Place remaining trees
        for _ in range(n - 1):
            placed_polygons = [p.polygon for p in placed_trees]
            tree_index = STRtree(placed_polygons)
            
            best_px, best_py, best_angle = None, None, 0
            min_radius = Decimal('Infinity')
            
            for _ in range(10):  # Try 10 positions for each tree
                tree_angle = random.uniform(0, 360)
                tree_to_place = ChristmasTree(angle=str(tree_angle))
                
                angle = generate_weighted_angle()
                vx = Decimal(str(math.cos(angle)))
                vy = Decimal(str(math.sin(angle)))
                
                radius = Decimal('20.0')
                step_in = Decimal('0.5')
                
                collision_found = False
                while radius >= 0:
                    px = radius * vx
                    py = radius * vy
                    
                    candidate_poly = affinity.translate(
                        tree_to_place.polygon,
                        xoff=float(px * scale_factor),
                        yoff=float(py * scale_factor))
                    
                    possible_indices = tree_index.query(candidate_poly)
                    if any((candidate_poly.intersects(placed_polygons[i]) and not
                            candidate_poly.touches(placed_polygons[i]))
                           for i in possible_indices):
                        collision_found = True
                        break
                    radius -= step_in
                
                if collision_found:
                    step_out = Decimal('0.05')
                    while True:
                        radius += step_out
                        px = radius * vx
                        py = radius * vy
                        
                        candidate_poly = affinity.translate(
                            tree_to_place.polygon,
                            xoff=float(px * scale_factor),
                            yoff=float(py * scale_factor))
                        
                        possible_indices = tree_index.query(candidate_poly)
                        if not any((candidate_poly.intersects(placed_polygons[i]) and not
                                   candidate_poly.touches(placed_polygons[i]))
                                   for i in possible_indices):
                            break
                else:
                    radius = Decimal('0')
                    px = Decimal('0')
                    py = Decimal('0')
                
                if radius < min_radius:
                    min_radius = radius
                    best_px = px
                    best_py = py
                    best_angle = tree_angle
            
            final_tree = ChristmasTree(center_x=str(best_px), center_y=str(best_py), angle=str(best_angle))
            placed_trees.append(final_tree)
        
        # Apply fix_direction
        optimized_side, best_rot_angle = optimize_rotation(placed_trees)
        if best_rot_angle != 0:
            placed_trees = apply_rotation(placed_trees, best_rot_angle)
        
        side = get_side_length(placed_trees)
        if side < best_side:
            best_side = side
            best_trees = [t.clone() for t in placed_trees]
    
    return best_trees, best_side

print("Build from scratch function defined")

In [ ]:
def aggressive_local_search(trees, max_iterations=200):
    """More aggressive local search with finer steps."""
    if len(trees) <= 1:
        return trees, get_side_length(trees)
    
    best_trees = [t.clone() for t in trees]
    best_side = get_side_length(best_trees)
    
    step_sizes = [0.05, 0.02, 0.01, 0.005, 0.002, 0.001, 0.0005]
    rotation_deltas = [15, 10, 5, 2, 1, 0.5, 0.2]
    
    for iteration in range(max_iterations):
        improved = False
        
        for tree_idx in range(len(best_trees)):
            original_tree = best_trees[tree_idx].clone()
            
            # Try translation moves
            for step in step_sizes:
                for dx, dy in [(step, 0), (-step, 0), (0, step), (0, -step),
                               (step, step), (step, -step), (-step, step), (-step, -step)]:
                    test_tree = ChristmasTree(
                        str(float(original_tree.center_x) + dx),
                        str(float(original_tree.center_y) + dy),
                        str(original_tree.angle)
                    )
                    
                    other_trees = best_trees[:tree_idx] + best_trees[tree_idx+1:]
                    if not has_collision(test_tree, other_trees):
                        test_trees = best_trees[:tree_idx] + [test_tree] + best_trees[tree_idx+1:]
                        test_side = get_side_length(test_trees)
                        
                        if test_side < best_side:
                            best_trees = test_trees
                            best_side = test_side
                            improved = True
                            break
                if improved:
                    break
            
            if improved:
                break
            
            # Try rotation moves
            for delta in rotation_deltas:
                for d_angle in [delta, -delta]:
                    test_tree = ChristmasTree(
                        str(original_tree.center_x),
                        str(original_tree.center_y),
                        str(float(original_tree.angle) + d_angle)
                    )
                    
                    other_trees = best_trees[:tree_idx] + best_trees[tree_idx+1:]
                    if not has_collision(test_tree, other_trees):
                        test_trees = best_trees[:tree_idx] + [test_tree] + best_trees[tree_idx+1:]
                        test_side = get_side_length(test_trees)
                        
                        if test_side < best_side:
                            best_trees = test_trees
                            best_side = test_side
                            improved = True
                            break
                if improved:
                    break
            
            if improved:
                break
        
        if not improved:
            break
    
    # Final fix_direction
    optimized_side, best_angle = optimize_rotation(best_trees)
    if optimized_side < best_side:
        best_trees = apply_rotation(best_trees, best_angle)
        best_side = optimized_side
    
    return best_trees, best_side

print("Aggressive local search defined")

In [ ]:
# Build configurations with aggressive optimization for small n
start_time = time.time()

all_configs = {}
all_side_lengths = {}

# For n=1, the optimal is just a single tree
all_configs[1] = [ChristmasTree(angle='45')]
all_side_lengths[1] = get_side_length(all_configs[1])
print(f"n=1: side={float(all_side_lengths[1]):.6f}")

# For small n (2-20), build from scratch with many attempts
for n in range(2, 21):
    trees, side = build_config_from_scratch(n, num_attempts=30)
    trees, side = aggressive_local_search(trees, max_iterations=100)
    all_configs[n] = trees
    all_side_lengths[n] = side
    
    if n % 5 == 0:
        elapsed = time.time() - start_time
        print(f"n={n}: side={float(side):.6f}, elapsed={elapsed:.1f}s")

print(f"\nSmall configs done. Elapsed: {time.time() - start_time:.1f}s")

In [ ]:
# For medium n (21-50), use incremental building with optimization
current_trees = all_configs[20]

for n in range(21, 51):
    # Add one tree to previous config
    placed_polygons = [p.polygon for p in current_trees]
    tree_index = STRtree(placed_polygons)
    
    best_px, best_py, best_angle = None, None, 0
    min_radius = Decimal('Infinity')
    
    for _ in range(30):  # Try 30 positions
        tree_angle = random.uniform(0, 360)
        tree_to_place = ChristmasTree(angle=str(tree_angle))
        
        angle = generate_weighted_angle()
        vx = Decimal(str(math.cos(angle)))
        vy = Decimal(str(math.sin(angle)))
        
        radius = Decimal('20.0')
        step_in = Decimal('0.5')
        
        collision_found = False
        while radius >= 0:
            px = radius * vx
            py = radius * vy
            
            candidate_poly = affinity.translate(
                tree_to_place.polygon,
                xoff=float(px * scale_factor),
                yoff=float(py * scale_factor))
            
            possible_indices = tree_index.query(candidate_poly)
            if any((candidate_poly.intersects(placed_polygons[i]) and not
                    candidate_poly.touches(placed_polygons[i]))
                   for i in possible_indices):
                collision_found = True
                break
            radius -= step_in
        
        if collision_found:
            step_out = Decimal('0.05')
            while True:
                radius += step_out
                px = radius * vx
                py = radius * vy
                
                candidate_poly = affinity.translate(
                    tree_to_place.polygon,
                    xoff=float(px * scale_factor),
                    yoff=float(py * scale_factor))
                
                possible_indices = tree_index.query(candidate_poly)
                if not any((candidate_poly.intersects(placed_polygons[i]) and not
                           candidate_poly.touches(placed_polygons[i]))
                           for i in possible_indices):
                    break
        else:
            radius = Decimal('0')
            px = Decimal('0')
            py = Decimal('0')
        
        if radius < min_radius:
            min_radius = radius
            best_px = px
            best_py = py
            best_angle = tree_angle
    
    new_tree = ChristmasTree(center_x=str(best_px), center_y=str(best_py), angle=str(best_angle))
    trees = [t.clone() for t in current_trees] + [new_tree]
    
    # Optimize
    trees, side = aggressive_local_search(trees, max_iterations=50)
    
    all_configs[n] = trees
    all_side_lengths[n] = side
    current_trees = trees
    
    if n % 10 == 0:
        elapsed = time.time() - start_time
        print(f"n={n}: side={float(side):.6f}, elapsed={elapsed:.1f}s")

print(f"\nMedium configs done. Elapsed: {time.time() - start_time:.1f}s")

In [ ]:
# For large n (51-200), use faster incremental building with less optimization
for n in range(51, 201):
    placed_polygons = [p.polygon for p in current_trees]
    tree_index = STRtree(placed_polygons)
    
    best_px, best_py, best_angle = None, None, 0
    min_radius = Decimal('Infinity')
    
    for _ in range(15):  # Fewer attempts for speed
        tree_angle = random.uniform(0, 360)
        tree_to_place = ChristmasTree(angle=str(tree_angle))
        
        angle = generate_weighted_angle()
        vx = Decimal(str(math.cos(angle)))
        vy = Decimal(str(math.sin(angle)))
        
        radius = Decimal('20.0')
        step_in = Decimal('0.5')
        
        collision_found = False
        while radius >= 0:
            px = radius * vx
            py = radius * vy
            
            candidate_poly = affinity.translate(
                tree_to_place.polygon,
                xoff=float(px * scale_factor),
                yoff=float(py * scale_factor))
            
            possible_indices = tree_index.query(candidate_poly)
            if any((candidate_poly.intersects(placed_polygons[i]) and not
                    candidate_poly.touches(placed_polygons[i]))
                   for i in possible_indices):
                collision_found = True
                break
            radius -= step_in
        
        if collision_found:
            step_out = Decimal('0.05')
            while True:
                radius += step_out
                px = radius * vx
                py = radius * vy
                
                candidate_poly = affinity.translate(
                    tree_to_place.polygon,
                    xoff=float(px * scale_factor),
                    yoff=float(py * scale_factor))
                
                possible_indices = tree_index.query(candidate_poly)
                if not any((candidate_poly.intersects(placed_polygons[i]) and not
                           candidate_poly.touches(placed_polygons[i]))
                           for i in possible_indices):
                    break
        else:
            radius = Decimal('0')
            px = Decimal('0')
            py = Decimal('0')
        
        if radius < min_radius:
            min_radius = radius
            best_px = px
            best_py = py
            best_angle = tree_angle
    
    new_tree = ChristmasTree(center_x=str(best_px), center_y=str(best_py), angle=str(best_angle))
    trees = [t.clone() for t in current_trees] + [new_tree]
    
    # Just fix_direction, no local search for speed
    optimized_side, best_rot_angle = optimize_rotation(trees)
    if best_rot_angle != 0:
        trees = apply_rotation(trees, best_rot_angle)
    
    all_configs[n] = trees
    all_side_lengths[n] = get_side_length(trees)
    current_trees = trees
    
    if n % 25 == 0:
        elapsed = time.time() - start_time
        print(f"n={n}: side={float(all_side_lengths[n]):.6f}, elapsed={elapsed:.1f}s")

print(f"\nAll configs done. Total time: {time.time() - start_time:.1f}s")

In [ ]:
# Calculate score
total_score = calculate_score(all_side_lengths)
print(f"Total Score: {total_score:.6f}")
print(f"Target: 68.922808")
print(f"Gap: {total_score - 68.922808:.6f}")

# Validate
invalid = [n for n, trees in all_configs.items() if has_any_overlap(trees)]
if invalid:
    print(f"WARNING: {len(invalid)} configs have overlaps: {invalid[:10]}...")
else:
    print("All configurations valid (no overlaps)")

In [ ]:
# Save submission
index = [f'{n:03d}_{t}' for n in range(1, 201) for t in range(n)]

tree_data = []
for n in range(1, 201):
    for tree in all_configs[n]:
        tree_data.append([float(tree.center_x), float(tree.center_y), float(tree.angle)])

cols = ['x', 'y', 'deg']
submission = pd.DataFrame(index=index, columns=cols, data=tree_data).rename_axis('id')

for col in cols:
    submission[col] = submission[col].astype(float).round(decimals=6)

for col in submission.columns:
    submission[col] = 's' + submission[col].astype('string')

submission.to_csv('/home/submission/submission.csv')
submission.to_csv('/home/code/experiments/004_aggressive_small/submission.csv')
print("Submission saved!")
print(f"\nFinal Score: {total_score:.6f}")